# Preparação de Dados (N1)

## 🗂️ [ Projeto ]

**Nome do Projeto:** [nome do projeto]  
**Dataset:** [Nome do dataset ou fonte de dados]  
**Objetivo Geral:** [Breve Explicação do propósito — ex.: “Analisar o comportamento de churn e identificar fatores de retenção de clientes.”]  

---

## 📋 Descrição Geral do Tratamento

Nesta etapa serão realizados os principais **processos de limpeza, transformação e padronização dos dados**.  
O foco é garantir que o dataset esteja **consistente, estruturado e pronto para análises posteriores**.

**Tarefas previstas:**  
- Leitura e diagnóstico inicial do dataset.  
- Tratamento de valores nulos, duplicados e outliers.  
- Conversão de tipos e padronização de nomes de colunas.  
- Codificação de variáveis categóricas e normalização de numéricas (quando necessário).  
- Geração de artefatos intermediários e relatório de qualidade dos dados.

## 🔧 Configuração do Projeto

Esta etapa prepara todo o ambiente do notebook antes de iniciar o tratamento dos dados.  
Ela garante que o projeto possa ser executado em qualquer máquina ou repositório com estrutura padronizada.  

### 📋 O que é feito aqui:

1. **Localização automática da raiz do projeto**  
   - Busca o arquivo `config/defaults.json` subindo diretórios até encontrá-lo.  
   - Define a variável `PROJECT_ROOT`, usada como referência para todos os caminhos seguintes.

2. **Validação e registro do pacote `utils/`**  
   - Verifica se existem os arquivos `utils/__init__.py` e `utils/utils_data.py`.  
   - Adiciona a raiz do projeto (`PROJECT_ROOT`) no `sys.path` para permitir importações de módulos internos.

3. **Importação das funções utilitárias**  
   - Carrega métodos de leitura, tratamento e salvamento de dados definidos no módulo `utils_data.py`.

4. **Carregamento de configurações globais**  
   - Lê os parâmetros de `config/defaults.json` e, se existir, substitui valores pelo `config/local.json`.  
   - Esses parâmetros controlam o comportamento de cada etapa (tratamento de nulos, outliers, encoding etc.).

5. **Definição de caminhos e diretórios principais**  
   - Gera variáveis globais para todos os diretórios padrão:  
     `data/raw`, `data/interim`, `data/processed`, `reports`, `artifacts`, `prints`, `dashboards`.  
   - Garante que todos existam, criando-os se necessário.  
   - Define também os arquivos principais de saída (`OUTPUT_INTERIM`, `OUTPUT_PROCESSED`).

6. **Configuração de ambiente de execução**  
   - Define uma semente aleatória (`RANDOM_SEED`) para reprodutibilidade.  
   - Ajusta opções de exibição do pandas (largura e número máximo de colunas).

7. **Inicialização do sistema de logs**  
   - Cria um log local em `reports/data_preparation.log`.  
   - Toda ação relevante (carregamento, conversão, limpeza, exportação etc.) será registrada nesse arquivo.

In [1]:
# -*- coding: utf-8 -*-
# Configurações do projeto (sem fallbacks; falha explícita se faltar algo)

from pathlib import Path
import sys, json, logging
import numpy as np
import pandas as pd
from typing import Dict, Any, Optional
from dataclasses import dataclass


# =============================================================================
# 1) Util: buscar caminho "subindo" diretórios
# =============================================================================
def find_upwards(relative_path: str, start: Optional[Path] = None) -> Optional[Path]:
    start = start or Path.cwd()
    rel_parts = Path(relative_path).parts
    for base in (start, *start.parents):
        candidate = base.joinpath(*rel_parts)
        if candidate.exists():
            return candidate
    return None

# =============================================================================
# 2) Descobrir raiz do projeto pela presença de config/defaults.json
# =============================================================================
_cfg_path = find_upwards("config/defaults.json")
if _cfg_path is None:
    raise FileNotFoundError(
        "config/defaults.json não encontrado. Crie a pasta 'config' na raiz do projeto "
        "e adicione o arquivo 'defaults.json' com as configurações."
    )
PROJECT_ROOT = _cfg_path.parent.parent.resolve()
print(f"[INFO] PROJECT_ROOT: {PROJECT_ROOT}")

# =============================================================================
# 3) Garantir sys.path e validar pacote utils/ 
# =============================================================================
UTILS_DIR = PROJECT_ROOT / "utils"
INIT_FILE = UTILS_DIR / "__init__.py"
UTILS_FILE = UTILS_DIR / "utils_data.py"

if not UTILS_DIR.exists():
    raise ModuleNotFoundError(f"Pasta não encontrada: {UTILS_DIR} (crie 'utils' na raiz do projeto).")
if not INIT_FILE.exists():
    raise ModuleNotFoundError(f"Arquivo não encontrado: {INIT_FILE} (crie um __init__.py vazio em 'utils').")
if not UTILS_FILE.exists():
    raise ModuleNotFoundError(f"Arquivo não encontrado: {UTILS_FILE} (adicione 'utils_data.py' dentro de 'utils').")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print(f"[INFO] sys.path ok. utils: {UTILS_DIR}")

# =============================================================================
# 4) Import das utilidades 
# =============================================================================
from utils.utils_data import (
    load_table_simple, infer_format_from_suffix, merge_chain,
    load_csv, save_parquet, basic_overview, reduce_memory_usage,
    infer_numeric_like, strip_whitespace, missing_report, simple_impute,
    detect_outliers_iqr, detect_outliers_zscore, deduplicate_rows,
    encode_categories, scale_numeric, save_table, simple_impute_with_flags,
    parse_dates_with_report, expand_date_features, build_calendar_from,
    encode_categories_safe, scale_numeric_safe, apply_encoding_and_scaling
)

# =============================================================================
# 5) Carregar configurações (defaults.json obrigatório; local.json opcional)
# =============================================================================
def load_config(base_abs: Path, local_abs: Optional[Path] = None) -> Dict[str, Any]:
    if not base_abs.exists():
        raise FileNotFoundError(f"Arquivo obrigatório não encontrado: {base_abs}")
    cfg = json.loads(base_abs.read_text(encoding="utf-8"))
    print(f"[INFO] Config carregada de: {base_abs}")
    if local_abs and local_abs.exists():
        local_cfg = json.loads(local_abs.read_text(encoding="utf-8"))
        cfg.update(local_cfg)
        print(f"[INFO] Overrides locais: {local_abs}")
    return cfg

CONFIG_DIR = PROJECT_ROOT / "config"
DEFAULTS_JSON = CONFIG_DIR / "defaults.json"
LOCAL_JSON = CONFIG_DIR / "local.json"
config: Dict[str, Any] = load_config(DEFAULTS_JSON, LOCAL_JSON)

# =============================================================================
# 6) Paths do projeto
# =============================================================================
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR = PROJECT_ROOT / "reports"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
PRINTS_DIR = PROJECT_ROOT / "prints"
DASHBOARDS_DIR = PROJECT_ROOT / "dashboards"
OUTPUT_INTERIM = INTERIM_DIR / "dataset_interim.csv"
OUTPUT_PROCESSED = PROCESSED_DIR / "dataset_processed.csv"

#OUTPUT_INTERIM_MAIN = INTERIM_DIR / "dataset_interim.csv"       # consolidado principal
#OUTPUT_INTERIM_CUSTOMERS = INTERIM_DIR / "customers_interim.csv" # tabela auxiliar
#OUTPUT_INTERIM_REGION = INTERIM_DIR / "region_interim.csv"       # outra fonte
#OUTPUT_PROCESSED = PROCESSED_DIR / "dataset_processed.csv"

for d in [RAW_DIR, INTERIM_DIR, PROCESSED_DIR, REPORTS_DIR, ARTIFACTS_DIR, PRINTS_DIR, DASHBOARDS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# =============================================================================
# 7) Seed & display
# =============================================================================
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)

# =============================================================================
# 8) Logging (console + arquivo)
# =============================================================================
LOG_FILE = REPORTS_DIR / 'data_preparation.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler(LOG_FILE, encoding='utf-8')]
)
logger = logging.getLogger(__name__)
logger.info('Project configuration loaded.')
logger.info(
    "Flags: missing=%s, outliers=%s(%s), encode=%s(%s), scale=%s(%s)",
    config.get("handle_missing"), config.get("detect_outliers"), config.get("outlier_method"),
    config.get("encode_categoricals"), config.get("encoding_type"),
    config.get("scale_numeric"), config.get("scaler")
)


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils
[INFO] Config carregada de: C:\Users\fabio\Projetos DEV\data projects\data-project-template\config\defaults.json
[INFO] Overrides locais: C:\Users\fabio\Projetos DEV\data projects\data-project-template\config\local.json
2025-10-28 08:45:38,762 | INFO | Project configuration loaded.
2025-10-28 08:45:38,762 | INFO | Flags: missing=True, outliers=True(iqr), encode=True(onehot), scale=True(minmax)


## 🔧 Configuração de Fontes

Nesta etapa são definidos os **arquivos de origem** que servirão de base para o projeto.  
Aqui é onde você informa **quais datasets serão carregados**, **em qual formato** estão (CSV ou Parquet) e, se houver mais de uma fonte, **como elas se relacionam**.

### 📋 O que é feito aqui:

1. **Definição das fontes de dados (`SOURCES`)**  
   - Cada entrada do dicionário representa uma tabela nomeada.  
   - A chave (ex.: `"main"`, `"dim_customers"`) será o identificador da tabela no projeto.  
   - Cada item contém:
     - `path`: caminho do arquivo no diretório `data/raw/`.  
     - `format`: formato opcional (`"csv"` ou `"parquet"`). Se omitido, o código identifica automaticamente pelo sufixo.  
     - `read_opts`: parâmetros de leitura (como `encoding`, `sep`, `low_memory` etc.).

   Exemplo de estrutura:
   ```python
   SOURCES = {
       "main": {
           "path": RAW_DIR / "dataset.csv",
           "read_opts": {"encoding": "utf-8", "sep": ","}
       },
       "dim_customers": {
           "path": RAW_DIR / "customers.parquet",
           "format": "parquet"
       }
   }


In [2]:
from utils.utils_data import list_directory_files

display(list_directory_files(RAW_DIR))

,Arquivo,Extensão,Tamanho (KB),Modificado em
0,.gitkeep,,0.0,2025-10-28 11:38:58
1,README.md,.md,0.2,2025-10-28 11:38:58
2,dataset.csv,.csv,954.6,2019-09-27 19:30:08


In [3]:
# Obs: só alterar este bloco quando precisar mudar os arquivos ou as chaves de junção.

SOURCES = {
    # nome_da_tabela: {path, (opcional) format: 'csv'|'parquet', (opcional) read_opts}
    "main": {
        "path": RAW_DIR / "dataset.csv",
        # "format": "csv",  # se omitir, detecta pelo sufixo
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo de segunda fonte em CSV:
    # "dim_customers": {
    #   "path": RAW_DIR / "exemplo.csv",
    #   "format": "csv",  # se omitir, detecta pelo sufixo
    #   "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    # }
    
    # Exemplo de fonte em Parquet:
    # "dim_customers": {
    #     "path": RAW_DIR / "customers.parquet",
    #     "format": "parquet"
    # }
}

# Plano de merges (opcional):
# Cada item é um passo: (right_name, how, left_on, right_on)
# O DataFrame base será o SOURCES[MAIN_SOURCE]
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]


## 📥 Ingestão & Visão Rápida

Nesta etapa ocorre o **carregamento efetivo dos datasets** definidos na configuração de fontes e a geração de uma **visão inicial dos dados**.  
O objetivo é confirmar se os arquivos foram lidos corretamente, identificar possíveis problemas e compreender a estrutura de cada tabela antes das transformações.

### 📋 O que é feito aqui

1. **Leitura das fontes (`SOURCES`)**
   - Cada fonte declarada na etapa anterior é carregada automaticamente.
   - O método `load_table_simple()` (presente em `utils/utils_data.py`) realiza a leitura:
     - Detecta o formato (`.csv` ou `.parquet`) com base na extensão, caso não seja informado.
     - Aplica parâmetros de leitura personalizados (`read_opts`) quando disponíveis.
   - Cada tabela é armazenada no dicionário `tables`, permitindo fácil acesso posterior.

2. **Geração de visão rápida por tabela**
   - Para cada dataset carregado são exibidos:
     - **Resumo geral** com número de linhas, colunas, tipos de dados e uso de memória (`basic_overview()`).
     - **Relatório de valores faltantes** ordenado pelas colunas mais afetadas (`missing_report()`).
   - Essa visualização inicial ajuda a detectar inconsistências, campos nulos ou formatos inesperados.

3. **Definição do DataFrame base (`df`)**
   - Seleciona a tabela principal definida em `MAIN_SOURCE`.
   - Caso existam etapas de junção configuradas (`MERGE_STEPS`), o código aplica os merges automaticamente via `merge_chain()`.
   - Se não houver junções, o DataFrame principal (`df`) segue inalterado para as próximas fases.

4. **Visão geral consolidada**
   - Após o merge (ou uso direto do dataset base), é exibido um novo resumo (`basic_overview`) e o relatório de faltantes atualizado.
   - Essas informações são registradas também no log do projeto, garantindo rastreabilidade.

> 💡 **Resumo:**  
> Esta célula realiza a **primeira análise estrutural dos dados**, validando se as fontes foram carregadas corretamente e consolidando o **DataFrame principal (`df`)** que será utilizado nas próximas etapas do pipeline (como limpeza, tipagem e tratamento de valores ausentes).


In [4]:
# 1) Carregar todas as fontes
tables = {}
for name, cfg in SOURCES.items():
    path = cfg["path"]
    fmt = cfg.get("format")              # se None, detecta pelo sufixo
    read_opts = cfg.get("read_opts", {}) # ex.: sep/encoding/low_memory para CSV
    df_src = load_table_simple(path, fmt, read_opts)  # utils.utils_data
    tables[name] = df_src

    # visão rápida por fonte
    print(f"\n=== {name} ===")
    ov = basic_overview(df_src)
    print(json.dumps(ov, indent=2, ensure_ascii=False))
    display(missing_report(df_src).head(20))

# 2) Definir df base e aplicar merges (se configurado)
if MAIN_SOURCE not in tables:
    raise KeyError(f"MAIN_SOURCE '{MAIN_SOURCE}' não encontrado. Fontes disponíveis: {list(tables.keys())}")

df = tables[MAIN_SOURCE]
if MERGE_STEPS:
    df = merge_chain(df, tables, MERGE_STEPS)  # utils.utils_data
    print("\n=== Visão geral (df merged) ===")
else:
    print(f"\n[INFO] Usando df base: '{MAIN_SOURCE}' (sem merges).")

# 3) Overview final do df que segue no pipeline
overview = basic_overview(df)
logger.info(json.dumps(overview, indent=2, ensure_ascii=False))
print(json.dumps(overview, indent=2, ensure_ascii=False))
display(missing_report(df).head(20))


2025-10-28 08:45:46,257 | INFO | Loading table: path=C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\raw\dataset.csv | format=csv | opts={'encoding': 'utf-8', 'sep': ',', 'low_memory': False}
2025-10-28 08:45:46,258 | INFO | Loading CSV: C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\raw\dataset.csv

=== main ===
{
  "shape": [
    7043,
    21
  ],
  "columns": [
    "customerID",
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges",
    "Churn"
  ],
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "Ph

,missing_rate,missing_count
customerID,0.0,0
DeviceProtection,0.0,0
TotalCharges,0.0,0
MonthlyCharges,0.0,0
PaymentMethod,0.0,0
PaperlessBilling,0.0,0
Contract,0.0,0
StreamingMovies,0.0,0
StreamingTV,0.0,0
TechSupport,0.0,0



[INFO] Usando df base: 'main' (sem merges).
2025-10-28 08:45:46,343 | INFO | {
  "shape": [
    7043,
    21
  ],
  "columns": [
    "customerID",
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges",
    "Churn"
  ],
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "PhoneService": "object",
    "MultipleLines": "object",
    "InternetService": "object",
    "OnlineSecurity": "object",
    "OnlineBackup": "object",
    "DeviceProtection": "object",
    "TechSupport": "object",
    "StreamingTV": "object",
    "StreamingMovies": "object",
    "C

,missing_rate,missing_count
customerID,0.0,0
DeviceProtection,0.0,0
TotalCharges,0.0,0
MonthlyCharges,0.0,0
PaymentMethod,0.0,0
PaperlessBilling,0.0,0
Contract,0.0,0
StreamingMovies,0.0,0
StreamingTV,0.0,0
TechSupport,0.0,0


## 🔖 Catálogo de DataFrames + df “ativo”

Esta etapa cria um **catálogo centralizado de DataFrames** para gerenciar facilmente todas as tabelas carregadas e derivadas ao longo do projeto.  
O objetivo é permitir que diferentes versões e transformações dos dados sejam armazenadas, nomeadas e acessadas de forma organizada e reprodutível.

### 📋 O que é feito aqui

1. **Inicialização do catálogo (`TableStore`)**
   - A classe `TableStore` (definida em `utils/utils_data.py`) atua como um **repositório de DataFrames nomeados**.  
   - Ela é inicializada com o dicionário `tables` (criado na etapa de ingestão) e define a tabela principal (`MAIN_SOURCE`) como **tabela ativa**.

2. **Definição do DataFrame ativo (`df`)**
   - A variável `df` recebe a tabela atual através de `T.get()`.  
   - Esse `df` será o **DataFrame padrão** para as próximas etapas do pipeline (limpeza, tipagem e transformação).

3. **Gerenciamento de múltiplas tabelas**
   - Novos DataFrames podem ser adicionados ao catálogo a qualquer momento, com nomes descritivos e controle de versão.
   - O catálogo também permite alternar entre tabelas e listar todas as disponíveis.

   **Principais comandos:**
   ```python
   T.add("churn_raw", df, set_current=True)           # adiciona e define como atual
   df = T.use("churn_raw")                            # alterna o df ativo
   T.add("features_v1", engenharia_de_atributos(df))  # armazena uma nova derivação
   df_features = T["features_v1"]                     # acesso direto por nome
   display(T.list())                                  # exibe o inventário completo
   
4. **Benefício prático**
   - Mantém o notebook limpo e evita sobrescrever DataFrames importantes.
   - Facilita o reuso e o rastreamento das versões intermediárias dos dados.
   - Ideal para projetos com múltiplas fontes, transformações paralelas ou comparações entre conjuntos tratados.

> 💡 **Resumo:**  
> Este catálogo funciona como uma memória estruturada de DataFrames, permitindo adicionar, alternar, versionar e consultar tabelas de forma controlada.
A variável (`df`) sempre representa a tabela ativa atual, garantindo consistência nas etapas subsequentes do pipeline.


In [5]:
from utils.utils_data import TableStore
# Inicializa catálogo com as fontes lidas ('tables') e define a base como atual
T = TableStore(initial=tables, current=MAIN_SOURCE)

# Conveniência: df = tabela atual (boa prática para seguir no pipeline)
df = T.get()

# Guia rápido (deixe como comentário para consulta):
# T.add("churn_raw", df, set_current=True)          # cadastra e ativa
# df = T.use("churn_raw")                           # muda o atual e retorna df
# T.add("features_v1", engenharia_de_atributos(df)) # salva uma derivação
# df_features = T["features_v1"]                    # acesso direto por nome
# display(T.list())                                 # inventário das tabelas


In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 🧪 Qualidade & Tipagem

Nesta etapa ocorre a **padronização estrutural e tipagem dos dados**, garantindo que o DataFrame principal (`df`) siga para as próximas fases com formatos consistentes, tipos adequados e menor consumo de memória.  
É uma das etapas mais importantes do pipeline, pois corrige inconsistências comuns antes das análises e modelagens.

### 📋 O que é feito aqui

1. **Remoção de espaços em branco (`strip_whitespace`)**
   - Elimina espaços extras no início e no fim de valores textuais.  
   - Evita divergências em comparações e agrupamentos (ex.: `"Yes "` ≠ `"Yes"`).

2. **Conversão de números em texto para valores numéricos (`infer_numeric_like`)**
   - Identifica colunas com valores aparentemente numéricos, mas armazenados como texto (ex.: `"1.234,56"` ou `"R$ 120,00"`).  
   - Converte automaticamente esses valores em números reais (`float`), respeitando símbolos e separadores regionais.
   - O comportamento é controlado pelos parâmetros:
     - `min_ratio`: proporção mínima de valores conversíveis (padrão: 0.9).  
     - `create_new_col_when_partial`: cria uma nova coluna (`col_num`) se apenas parte dos valores for conversível.  
     - `blacklist`: impede que colunas específicas (ex.: IDs) sejam convertidas.  
   - Gera um relatório (`cast_report`) exibindo:
     - Coluna analisada  
     - Ação executada (convertida, ignorada, parcial)  
     - Taxa de conversão e exemplos de valores não convertidos.

3. **Otimização de tipos numéricos (`reduce_memory_usage`)**
   - Reduz o uso de memória ao substituir tipos numéricos genéricos (`int64`, `float64`) por versões mais leves (`int32`, `float32`, etc.).  
   - Essa otimização é transparente e não altera os valores.  
   - Garante maior eficiência e velocidade nas operações futuras.

4. **Exportação do dataset “interim” (`save_table`)**
   - Salva o DataFrame tratado no formato definido por `OUTPUT_INTERIM` (`.csv` ou `.parquet`).  
   - Esse arquivo representa a versão intermediária, limpa e padronizada, que servirá de entrada para as próximas fases do projeto.

> 💡 **Resumo:**  
> Esta célula garante que os dados estejam **limpos, tipados e otimizados**.  
> Corrige formatações, converte valores textuais em numéricos, reduz o consumo de memória e salva uma cópia organizada dos dados — preparando o terreno para as próximas etapas do pipeline.


In [7]:
# 1) Texto: espaços em branco
if config.get("strip_whitespace", True):
    df = strip_whitespace(df)
    logger.info("[N1] strip_whitespace aplicado.")

# 2) Números em texto → numérico 
if config.get("cast_numeric_like", True):
    df, cast_report = infer_numeric_like(
        df,
        columns=None,                  # listar colunas específicas
        min_ratio=0.9,
        create_new_col_when_partial=True,
        blacklist=["customerID"]       # ids nunca devem virar número
    )
    display(cast_report)
    logger.info("[N1] infer_numeric_like aplicado.")

# 3) Otimização / downcast de tipos numéricos
if config.get("infer_types", True):
    df = reduce_memory_usage(df)
    logger.info("[N1] reduce_memory_usage aplicado.")

# 4) Exporta interim
if config.get("export_interim", True):
    save_table(df, OUTPUT_INTERIM)
    logger.info(f"[N1] Exportado interim: {OUTPUT_INTERIM}")

# 5) Duplicidades
if config.get("deduplicate", True):
    subset = config.get("deduplicate_subset") or None
    keep = config.get("deduplicate_keep", "first")
    log_path = None
    if config.get("deduplicate_log", True):
        fname = config.get("deduplicate_log_filename", "duplicates.csv")
        log_path = REPORTS_DIR / fname

    df, dups_df, dup_summary = deduplicate_rows(
        df,
        subset=subset,
        keep=keep,
        log_path=log_path,
        return_report=True
    )

    # feedback visual
    if not dups_df.empty:
        print("🔁 Linhas duplicadas detectadas (amostra):")
        display(dups_df.head(10))
        print("📊 Resumo por chave:")
        display(dup_summary.head(20))
    else:
        print("✅ Nenhuma duplicidade encontrada segundo os critérios definidos.")


2025-10-28 08:45:49,828 | INFO | [N1] strip_whitespace aplicado.
2025-10-28 08:45:50,086 | INFO | [infer_numeric_like] summary:
          column             action  ratio  converted  non_convertible                                                    examples
    TotalCharges    apply_overwrite    1.0       7032                0                                                          []
           Churn skip_no_conversion    0.0          0             7043                                                   [No, Yes]
        Contract skip_no_conversion    0.0          0             7043                        [Month-to-month, One year, Two year]
      Dependents skip_no_conversion    0.0          0             7043                                                   [No, Yes]
DeviceProtection skip_no_conversion    0.0          0             7043                              [No, Yes, No internet service]
 InternetService skip_no_conversion    0.0          0             7043                

,column,action,ratio,converted,non_convertible,examples
0,TotalCharges,apply_overwrite,1.0,7032,0,[]
1,Churn,skip_no_conversion,0.0,0,7043,"[No, Yes]"
2,Contract,skip_no_conversion,0.0,0,7043,"[Month-to-month, One year, Two year]"
3,Dependents,skip_no_conversion,0.0,0,7043,"[No, Yes]"
4,DeviceProtection,skip_no_conversion,0.0,0,7043,"[No, Yes, No internet service]"
5,InternetService,skip_no_conversion,0.0,0,7043,"[DSL, Fiber optic, No]"
6,MultipleLines,skip_no_conversion,0.0,0,7043,"[No phone service, No, Yes]"
7,OnlineBackup,skip_no_conversion,0.0,0,7043,"[Yes, No, No internet service]"
8,OnlineSecurity,skip_no_conversion,0.0,0,7043,"[No, Yes, No internet service]"
9,PaperlessBilling,skip_no_conversion,0.0,0,7043,"[Yes, No]"


2025-10-28 08:45:50,108 | INFO | [N1] infer_numeric_like aplicado.
2025-10-28 08:45:50,138 | INFO | Memory reduced: 6.51MB -> 6.37MB
2025-10-28 08:45:50,138 | INFO | [N1] reduce_memory_usage aplicado.
2025-10-28 08:45:50,171 | INFO | Saved: C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\interim\dataset_interim.csv
2025-10-28 08:45:50,171 | INFO | [N1] Exportado interim: C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\interim\dataset_interim.csv
2025-10-28 08:45:50,189 | INFO | [deduplicate] Removed duplicates: 0 (subset=None, keep=first)
✅ Nenhuma duplicidade encontrada segundo os critérios definidos.


## 🧼 Padronização Categórica

Esta etapa realiza uma **limpeza leve e explícita** nas colunas categóricas do dataset, garantindo que valores equivalentes sejam tratados de forma uniforme.  
É uma forma simples de **padronizar rótulos textuais** antes de etapas posteriores como imputação, codificação ou engenharia de atributos.

---

### 📋 O que é feito aqui
Unificar rótulos redundantes ou inconsistentes que representam o mesmo conceito.

1. **Identificação de colunas categóricas** (`object` ou `category`);  
2. **Limpeza de forma** – remoção de espaços extras e normalização de espaçamento;  
3. **Substituição direta** com base no dicionário `replacements`;  
4. **Registro em log** das colunas que foram alteradas.

Essa normalização reduz o número de categorias distintas, evitando ruído e inconsistências em análises e modelos.

---

### 🧩 Como personalizar
- O dicionário `replacements` é **manual e transparente**: edite conforme o contexto do projeto.  
  Exemplo:
  ```python
  replacements = {
        "No internet service": "No",
        "No phone service": "No",
        "n/a": "No",
        "none": "No",
    }

> 💡 **Resumo:**  
> Esta célula padroniza valores categóricos redundantes de forma **simples, explícita e controlada**, garantindo consistência nos dados **sem aumentar a complexidade do pipeline**.

In [8]:
if config.get("normalize_categories", True):
    replacements = {
        "No internet service": "No",
        "No phone service": "No",
        "n/a": "No",
        "none": "No",
    }

    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    changed = []

    for col in cat_cols:
        before = df[col].copy()
        # limpeza leve de forma
        s = df[col].astype("string").str.strip().str.replace(r"\s+", " ", regex=True)
        # normalização explícita (mapa simples)
        s = s.replace(replacements)
        df[col] = s

        if not df[col].equals(before):
            changed.append(col)

    if changed:
        logger.info(f"[N1] Padronização categórica aplicada em: {changed}")
    else:
        logger.info("[N1] Padronização categórica: nenhuma alteração necessária.")


2025-10-28 08:45:51,125 | INFO | [N1] Padronização categórica aplicada em: ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']


## 🩹 Tratamento de Valores Faltantes

Nesta etapa são tratadas as **ausências de dados** (valores nulos ou `NaN`) para garantir que o DataFrame siga consistente para as próximas fases do pipeline.  
O objetivo é **preencher ou sinalizar valores faltantes** de forma controlada, preservando a integridade estatística das colunas e registrando quais linhas foram modificadas.

### 📋 O que é feito aqui

1. **Geração de relatório inicial**
   - Exibe um diagnóstico das colunas com valores ausentes, mostrando a proporção (`missing_rate`) e o total (`missing_count`) de registros nulos.
   - Essa visualização ajuda a decidir se o tratamento será simples ou se exigirá técnicas específicas.

2. **Aplicação da estratégia de imputação**
   - O comportamento é controlado pela chave `missing_strategy` do arquivo de configuração:
     - `"simple"` → aplica a função `simple_impute_with_flags()`
     - `"advanced"` → reservado para métodos personalizados (ex.: KNN, regressão, interpolação)
   - A estratégia **simples** executa:
     - Substituição de valores nulos **numéricos** pela **mediana** da coluna.  
     - Substituição de valores nulos **categóricos** pela **moda** (valor mais frequente).  
   - Durante a imputação, são criadas **colunas de flag** no formato `was_imputed_<coluna>`, indicando quais linhas foram alteradas.

3. **Relatório final de verificação**
   - Após o preenchimento, é exibido novamente o relatório de faltantes para confirmar se todas as lacunas foram resolvidas.
   - Caso ainda existam colunas críticas, o notebook pode ser ajustado para aplicar métodos mais avançados.

### Exemplo de flag gerada:
| customerID | TotalCharges | was_imputed_TotalCharges |
|-------------|--------------|--------------------------|
| 7590-VHVEG  | 29.85        | False                   |
| 9237-HQITU  | 1840.75      | True                    |

Linhas com `True` indicam que o valor original estava ausente e foi imputado automaticamente.

> 💡 **Resumo:**  
> Esta célula identifica e corrige valores faltantes, aplicando regras de imputação simples e registrando onde cada substituição ocorreu.  
> Isso garante **transparência, rastreabilidade e controle de qualidade**, permitindo filtrar posteriormente apenas os dados considerados consistentes.


In [9]:
if config.get("handle_missing", True):
    print("Relatório de faltantes (antes):")
    display(missing_report(df).head(20))

    if config.get("missing_strategy") == "simple":
        df = simple_impute_with_flags(df)
    else:
        # espaço para técnicas avançadas
        pass

    print("Relatório de faltantes (depois):")
    display(missing_report(df).head(20))


Relatório de faltantes (antes):


,missing_rate,missing_count
TotalCharges,0.001562,11
customerID,0.000000,0
DeviceProtection,0.000000,0
MonthlyCharges,0.000000,0
PaymentMethod,0.000000,0
PaperlessBilling,0.000000,0
Contract,0.000000,0
StreamingMovies,0.000000,0
StreamingTV,0.000000,0
TechSupport,0.000000,0


2025-10-28 08:45:52,048 | INFO | [impute] coluna 'TotalCharges' → 11 valores preenchidos (mediana).
Relatório de faltantes (depois):


,missing_rate,missing_count
customerID,0.0,0
gender,0.0,0
Churn,0.0,0
TotalCharges,0.0,0
MonthlyCharges,0.0,0
PaymentMethod,0.0,0
PaperlessBilling,0.0,0
Contract,0.0,0
StreamingMovies,0.0,0
StreamingTV,0.0,0


## 🚩 Detecção de Outliers

Esta etapa identifica **valores atípicos** nas colunas numéricas, adicionando colunas de flag (`*_is_outlier`) que indicam quais registros se desviam do padrão estatístico esperado.  
Essas colunas servem para **inspeção e auditoria**, sem alterar ou remover dados — a decisão de tratamento posterior (ajuste, exclusão ou manutenção) é **de negócio**.

### ⚙️ Como funciona

1. **Verifica a configuração**
   - A execução depende da flag `detect_outliers` no arquivo `config/defaults.json`.  
     Se estiver definida como `true`, a detecção é realizada.

2. **Seleciona o método estatístico**
   - `"iqr"` → usa o **Intervalo Interquartil (Interquartile Range)**:
     - Calcula Q1 (25º percentil) e Q3 (75º percentil)
     - Define limites:  
       Inferior = Q1 − 1.5 × IQR  
       Superior = Q3 + 1.5 × IQR
     - Valores fora desse intervalo são marcados como outliers.
   - `"zscore"` → usa o **desvio-padrão (Z-Score)**:
     - Calcula a média (μ) e o desvio-padrão (σ)
     - Para cada valor, obtém `z = (x − μ) / σ`
     - Valores com |z| > 3 são considerados outliers.

3. **Criação de colunas de flag**
   - Para cada variável numérica analisada, é criada uma nova coluna:
     ```
     <coluna>_is_outlier
     ```
   - O valor será:
     - `True` → registro identificado como outlier  
     - `False` → registro dentro do intervalo esperado

4. **Registro no log**
   - Ao final, o sistema registra no log quantas colunas de flag foram criadas:
     ```
     Outlier flags created: 4
     ```
     Significa que 4 colunas numéricas foram analisadas e receberam suas respectivas flags.

### Exemplo de resultado

| tenure | MonthlyCharges | TotalCharges | tenure_is_outlier | MonthlyCharges_is_outlier | TotalCharges_is_outlier |
|--------|----------------|--------------|-------------------|----------------------------|--------------------------|
| 5      | 35.50          | 190.00       | False             | False                      | False                    |
| 72     | 120.00         | 9999.99      | False             | **True**                   | **True**                 |

> 💡 **Resumo:**  
> Esta célula adiciona colunas de marcação para detectar valores atípicos de acordo com o método configurado.  
> Os dados originais são preservados, permitindo que a análise posterior defina se esses registros devem ser **mantidos, ajustados ou removidos**.


In [10]:
if config['detect_outliers']:
    if config['outlier_method'] == 'iqr':
        df = detect_outliers_iqr(df)
    elif config['outlier_method'] == 'zscore':
        df = detect_outliers_zscore(df)
    else:
        raise ValueError('Unknown outlier method.')

    outlier_cols = [c for c in df.columns if c.endswith('_is_outlier')]
    logger.info(f'Outlier flags created: {len(outlier_cols)}')

2025-10-28 08:45:52,974 | INFO | Outlier flags created: 4


In [11]:
display(df.filter(like="_is_outlier").sum())

SeniorCitizen_is_outlier     1142
tenure_is_outlier               0
MonthlyCharges_is_outlier       0
TotalCharges_is_outlier         0
dtype: Int64

## 🧬 Duplicidades

Remove **linhas duplicadas** do DataFrame para evitar contagens infladas, vieses e ruído em métricas.

### 📋 O que acontece aqui
- Por padrão, a duplicidade é verificada **linha a linha** (todas as colunas iguais).  
- A primeira ocorrência é mantida e as demais são removidas.

### 🔧 Opções (se configuradas no `config`)
- `deduplicate_subset`: lista de colunas que definem a chave de deduplicação  
  *Ex.:* `["customerID"]` mantém apenas um registro por cliente.
- `deduplicate_keep`: política de retenção — `"first"`, `"last"` ou `false` (remove todas as repetições).
- `deduplicate_log` + `deduplicate_log_filename`: salva um **CSV** com as duplicatas detectadas em `reports/`.

### Boas práticas:
- Uso do `subset` quando a duplicidade for **conceitual** (ex.: mesma pessoa/pedido), não necessariamente toda a linha idêntica.
- Gerar e revisar o relatório de duplicatas antes de decidir pela remoção definitiva.

> 💡**Resumo:** esta etapa garante um dataset **sem registros repetidos** segundo o critério definido, mantendo rastreabilidade quando o log estiver habilitado.

In [12]:
if config['deduplicate']:
    df = deduplicate_rows(df)

2025-10-28 08:45:54,374 | INFO | [deduplicate] Removed duplicates: 0 (subset=None, keep=first)


## 🛠️ Engenharia de Atributos

Criação de novas colunas que capturem **relações, proporções, categorias ou padrões** relevantes ao negócio.  
Essa etapa é inteiramente manual e depende do contexto do dataset.

### 💡 Exemplos:
- **Razões e proporções:** `TotalCharges / tenure` → gasto médio mensal.  
- **Flags categóricas:** `Contract == 'Month-to-month'` → 1 se contrato mensal.  
- **Contagens de serviços:** soma de colunas binárias (Yes/No).  

Essas novas features tornam as análises mais ricas e **melhoram a performance de modelos preditivos**.


In [13]:
if config['feature_engineering']:
    # Exemplo genérico (comente/remova conforme o caso):
    # if {'col_a','col_b'}.issubset(df.columns):
    #     df['a_per_b'] = df['col_a'] / df['col_b'].replace(0, np.nan)
    pass

# 📅 Tratamento de Datas

Esta etapa detecta e converte colunas de datas de forma **automática e controlada**, criando **features temporais** úteis para análises e modelos preditivos.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas de data

O sistema procura automaticamente colunas com nomes que contenham termos como  
`date`, `data`, `dt_`, `_dt`, ou `_date`.

Também é possível **forçar colunas específicas** definindo manualmente em:

```python
date_cfg["explicit_cols"] = ["StartDate", "EndDate"]
```

### 🔹 Conversão para datetime com auditoria

Cada coluna candidata é testada com diferentes formatos e tentativas de *parsing*.  
O relatório **`parse_report`** mostra:

| column      | parsed_ratio | converted |
|--------------|--------------|------------|
| order_date   | 1.00         | True       |
| start_date   | 0.35         | False      |

- **column:** nome da coluna testada  
- **parsed_ratio:** porcentagem de valores convertidos com sucesso  
- **converted:** indica se a coluna foi convertida (baseado no `min_ratio`)

---

### 🔹 Criação automática de features temporais

Para cada coluna convertida em `datetime`, são geradas variáveis derivadas como:

`*_year`, `*_month`, `*_day`, `*_dayofweek`, `*_quarter`, `*_week`,  
`*_is_month_start`, `*_is_month_end`

O prefixo das novas colunas é o **nome original da coluna de data** .

---

### 🧱 Comportamento defensivo

Caso nenhuma coluna seja detectada, a célula não gera erro — apenas loga:

`[dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.`


Assim, o pipeline segue normalmente.

---

### ⚙️ Configuração (`date_cfg`)

| Parâmetro       | Descrição                                                        | Exemplo               |
|-----------------|------------------------------------------------------------------|------------------------|
| `detect_regex`  | Padrão para localizar colunas com nomes de data                 | `"date"`              |
| `explicit_cols` | Lista manual de colunas a converter                             | `["StartDate"]`       |
| `dayfirst`      | Define se o formato é D/M/Y                                     | `True` para 🇧🇷        |
| `utc`           | Define se a conversão deve ser em UTC                           | `False`               |
| `formats`       | Lista de formatos específicos                                   | `["%d/%m/%Y"]`        |
| `min_ratio`     | Fração mínima de parsing bem-sucedido para aceitar a conversão  | `0.8`                 |

---

> 💡 **Resumo:**  
> Esta célula realiza o reconhecimento automático de colunas de data,
converte-as para o formato datetime e gera variáveis derivadas como
ano, mês, dia e semana.
> Caso nenhuma coluna de data seja encontrada, o código é ignorado com segurança,
garantindo a continuidade do pipeline sem erros.
> Os parâmetros em date_cfg permitem ajustar formato, localização e tolerância
de parsing conforme a estrutura de cada dataset.

In [14]:
date_cfg = {
  "detect_regex": r"(date|data|dt_|_dt$|_date$)",
  "explicit_cols": [],   # ex.: ["StartDate","EndDate"]
  "dayfirst": False,     # True se datas forem D/M/Y
  "utc": False,
  "formats": [],         # ex.: ["%d/%m/%Y", "%Y-%m-%d"]
  "min_ratio": 0.80,
}


# Converter e auditar parsing
df, parse_report, parsed_cols = parse_dates_with_report(df, date_cfg)
display(parse_report)

if not parsed_cols:
    logger.info("[dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.")
else:
    created = expand_date_features(
        df, parsed_cols,
        features=["year","month","day","dayofweek","quarter","week","is_month_start","is_month_end"],
        prefix_mode="auto"
    )
    logger.info(f"[dates] features criadas: {len(created)}")

2025-10-28 08:45:56,115 | INFO | [dates] candidates=[]
2025-10-28 08:45:56,116 | INFO | [dates] parsed_ok=[]


,column,parsed_ratio,converted


2025-10-28 08:45:56,120 | INFO | [dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.


# 🗓️ Criação da Tabela Calendário (`dim_date`)

Esta etapa gera automaticamente uma **tabela calendário completa** — também chamada de **dimensão de tempo** — a partir de uma coluna de datas existente no dataset principal.  
A tabela é útil para **análises temporais, dashboards e modelos de previsão** que utilizam períodos como referência (ano, mês, trimestre, etc).

---

## 📋 O que acontece aqui

### 🔹 Seleção da coluna de data

Você define manualmente qual coluna será usada como base:

```python
CAL_DATE_COL = "order_date"  # nome da coluna datetime escolhida
CAL_FREQ = "D"               # frequência: "D" (diário), "W" (semanal), "M" (mensal)
```

A coluna deve estar no formato **datetime**.  
Se não estiver, será exibido um erro pedindo para executar antes a etapa de **Tratamento de Datas**.

---

### 🔹 Geração da dimensão calendário

A função `build_calendar_from()` constrói uma tabela com todas as datas entre o **mínimo** e o **máximo** encontrados em `CAL_DATE_COL`.

Para cada data, são criadas colunas derivadas, como:

| Coluna          | Descrição                                |
|------------------|-------------------------------------------|
| `date`           | Data base (chave principal)              |
| `year`           | Ano                                      |
| `month`          | Mês numérico                             |
| `day`            | Dia do mês                               |
| `quarter`        | Trimestre (1–4)                          |
| `week`           | Semana do ano (ISO)                      |
| `dow`            | Dia da semana (0 = segunda, 6 = domingo) |
| `is_month_start` | Indica se a data é o primeiro dia do mês |
| `is_month_end`   | Indica se a data é o último dia do mês   |
| `month_name`     | Nome do mês                              |
| `day_name`       | Nome do dia da semana                    |

---

### 🔹 Armazenamento e reuso

A tabela é salva automaticamente no diretório de artefatos (`artifacts/`):

```python
CAL_OUT = ARTIFACTS_DIR / "dim_date.csv"
```

O formato é determinado pela extensão (.csv ou .parquet).
Além disso, a tabela pode ser registrada no catálogo de DataFrames (T) para uso posterior no pipeline:
```python
T.add("dim_date", dim_date)
```
> 💡 **Resumo:**  
> Esta célula cria uma dimensão de tempo completa baseada na coluna de data escolhida.
> Ela facilita comparações e agregações por ano, mês, semana ou trimestre, além de permitir junções temporais consistentes com outros datasets ou dashboards (ex.: Power BI).
> O processo é automático, reproduzível e independente do dataset principal.

In [15]:
# Ajuste estes parâmetros conforme o dataset:
CAL_DATE_COL = "order_date"           # escolha aqui a coluna de datas já convertida para datetime
CAL_FREQ     = "D"                    # "D" (diário), "W" (semanal), "M" (mensal) etc.
CAL_OUT      = ARTIFACTS_DIR / "dim_date.csv"  # caminho de saída (csv/parquet conforme a extensão)

# --- validações suaves ---
if CAL_DATE_COL not in df.columns:
    msg = f"⚠️ Coluna '{CAL_DATE_COL}' não encontrada no DataFrame. " \
          f"Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário."
    print(msg)
    logger.warning(msg)
else:
    if not pd.api.types.is_datetime64_any_dtype(df[CAL_DATE_COL]):
        msg = f"⚠️ Coluna '{CAL_DATE_COL}' existe, mas **não está em formato datetime**. " \
              "Execute a etapa de Tratamento de Datas antes, ou converta manualmente."
        print(msg)
        logger.warning(msg)
    else:
        # --- construção da dimensão calendário ---
        dim_date = build_calendar_from(df, CAL_DATE_COL, freq=CAL_FREQ)

        # --- visão rápida ---
        display(dim_date.head(12))
        start_date = dim_date["date"].min()
        end_date   = dim_date["date"].max()
        print(f"Período: {start_date.date()} → {end_date.date()}  | Linhas: {len(dim_date)}")

        # --- salvar em disco respeitando a extensão do caminho ---
        save_table(dim_date, CAL_OUT)
        logger.info(f"[calendar] Tabela calendário salva em: {CAL_OUT}")

        # --- opcional: registrar no catálogo de tabelas ---
        try:
            T.add("dim_date", dim_date)
            logger.info("[calendar] 'dim_date' registrada no catálogo T.")
        except NameError:
            # Se TableStore (T) não estiver sendo usado nesta sessão, ignore.
            pass


⚠️ Coluna 'order_date' não encontrada no DataFrame. Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário.
2025-10-28 08:45:57,452 | WARNING | ⚠️ Coluna 'order_date' não encontrada no DataFrame. Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário.


# 📝 Tratamento de Texto (opcional)

Esta etapa extrai **métricas numéricas e lógicas a partir de colunas textuais**, transformando texto livre em informações quantitativas úteis para **análise exploratória e modelagem**.  

É uma forma leve e controlada de **estruturar dados não numéricos**, sem recorrer a técnicas avançadas de NLP.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas textuais

O sistema busca automaticamente colunas com tipo `object` e ignora aquelas listadas na *blacklist*:

```python
text_cols = [c for c in df.columns if df[c].dtype == 'object' and c not in TEXT_CFG["blacklist"]]
```

Essas colunas normalmente contêm informações como:  
descrições, comentários, nomes, categorias textuais ou observações.

---

### 🔹 Limpeza e padronização

Antes de gerar as métricas, os textos passam por uma limpeza leve:
- Remoção de **espaços duplicados** e **trim** nas extremidades.  
- Conversão para **minúsculas**, garantindo consistência em análises de termos.

Essas ações são controladas pelas chaves:
```python
"lower": True,
"strip_collapse_ws": True
```

---

### 🔹 Criação automática de métricas textuais

Para cada coluna textual, são geradas novas colunas numéricas:

| Nova Coluna           | Descrição | Exemplo ("This is great!") |
|------------------------|------------|-----------------------------|
| `<coluna>_len`         | Número total de caracteres no texto | 15 |
| `<coluna>_word_count`  | Número de palavras separadas por espaços | 3 |
| `<coluna>_alpha_count` | Número de letras (A–Z, a–z) | 13 |
| `<coluna>_digit_count` | Número de dígitos numéricos | 0 |

---

### 🔹 Presença de termos-chave

O sistema também cria colunas booleanas (`True` / `False`) para identificar a **ocorrência de palavras específicas** em cada coluna textual.  

Exemplo:
```python
TEXT_CFG["keywords"] = ["error", "cancel", "premium"]
```

Gera colunas como:
- `<coluna>_has_error`  
- `<coluna>_has_cancel`  
- `<coluna>_has_premium`

Essas variáveis são úteis para análises de sentimento ou padrões de ocorrência em feedbacks de clientes.

---

### 🔹 Exportação de resumo

Ao final, é gerado um **resumo CSV** com as colunas de texto processadas e suas features derivadas.  
O arquivo é salvo em:
```
reports/text_features/summary.csv
```

Esse relatório documenta as transformações aplicadas, garantindo **rastreabilidade e transparência**.

---

### ⚙️ Configuração (`TEXT_CFG`)

| Parâmetro              | Descrição                                                       | Exemplo                              |
|------------------------|------------------------------------------------------------------|--------------------------------------|
| `lower`                | Converte o texto para minúsculas                                | `True`                               |
| `strip_collapse_ws`    | Remove espaços duplicados e limpa extremidades                  | `True`                               |
| `keywords`             | Lista de termos a detectar no texto                             | `["error", "cancel", "premium"]`     |
| `blacklist`            | Colunas a ignorar durante o processamento                       | `["customerID"]`                     |
| `export_summary`       | Salva relatório com resumo das features geradas                 | `True`                               |

---

### Boas práticas

- Aplicar esta etapa apenas em colunas realmente textuais — evite IDs ou códigos.  
- Personalizar a lista de **palavras-chave** conforme o contexto do dataset.  
- Caso o dataset não possua colunas de texto, o processo será ignorado com segurança.  
- Utilize o arquivo de resumo (`summary.csv`) para acompanhar colunas derivadas e auditorias.

---

> 💡 **Resumo:**  
> Esta célula transforma campos de texto em **indicadores numéricos e lógicos**, gerando métricas básicas (tamanho, palavras, letras, dígitos) e flags de presença de termos-chave.  
> Tudo é processado automaticamente com **configuração leve e reprodutível**, integrando dados textuais ao pipeline de forma organizada e escalável.


In [16]:
TEXT_CFG = {
    "lower": True,
    "strip_collapse_ws": True,
    "keywords": ["error", "cancel", "premium"],
    "blacklist": ["customerID"],
    "export_summary": True,
}

if config.get("text_features", True):
    df, text_summary = extract_text_features(
        df,
        lower=TEXT_CFG["lower"],
        strip_collapse_ws=TEXT_CFG["strip_collapse_ws"],
        keywords=TEXT_CFG["keywords"],
        blacklist=TEXT_CFG["blacklist"],
        export_summary=TEXT_CFG["export_summary"],
        summary_dir=REPORTS_DIR / "text_features"
    )
    display(text_summary.head())


NameError: name 'extract_text_features' is not defined

# 🔤 Codificação de Categóricas & 🔢 Escalonamento Numérico (opcionais)

Esta etapa transforma variáveis **categóricas** em representações numéricas e **padroniza a escala** das variáveis **numéricas** quando necessário.  
É útil para alimentar modelos de ML que **não aceitam strings** (ex.: regressões, SVMs, redes neurais) e/ou **são sensíveis à escala** (KNN, SVM com kernel RBF, PCA, etc.).

---

### 🧭 Visão Geral

- **Codificação (Encoding)**
  - **One-hot**: cria uma coluna por categoria (0/1). É a opção **mais segura** para baseline; evita ordenar categorias artificialmente.
  - **Ordinal**: mapeia categorias para números inteiros. Mais **compacto**, mas induz **ordem artificial**; use com cuidado.

- **Escalonamento (Scaling)**
  - **Standard**: `z = (x - média) / desvio`; centra em 0 com variância ≈ 1. Bom para dados **aprox. gaussianos**.
  - **MinMax**: escala para **[0, 1]**; útil quando limites mínimos e máximos são relevantes (redes neurais, normalizações simples).

---

### ⚙️ Configuração usada no notebook

Estas configurações são lidas do `config/defaults.json` (com `local.json` sobrepondo, se existir) e são repassadas como dicionários para a função `apply_encoding_and_scaling`:

```python
ENCODE_CFG = {
    "enabled":           config.get("encode_categoricals", True),
    "type":              config.get("encoding_type", "onehot"),  # "onehot" | "ordinal"
    "exclude_cols":      ["Churn", "customerID"],                # NÃO codificar alvo/ids
    "high_card_threshold": 50,                                    # ignora colunas com cardinalidade muito alta
}

SCALE_CFG = {
    "enabled":           config.get("scale_numeric", False),
    "method":            config.get("scaler", "standard"),       # "standard" | "minmax"
    "exclude_cols":      ["Churn"],                              # NÃO escalar alvo
    "only_continuous":   True,                                   # evita escalar dummies e inteiros-discretos
}

df, encoding_meta, scaling_meta = apply_encoding_and_scaling(
    df, encode_cfg=ENCODE_CFG, scale_cfg=SCALE_CFG
)
```

> 💡 **Dica:** deixe `scale_numeric = false` no início do projeto (exploração/entendimento). Ative a escala somente quando partir para a **modelagem** e **validação**.

---

### 🧩 O que a função faz (alto nível)

1. **Seleciona colunas categóricas** (`object`/`category`) **excluindo** as listadas em `exclude_cols` e as com **cardinalidade > high_card_threshold** (proteção contra explosão de dummies).
2. **Codifica** conforme `type`:
   - `onehot` → gera `get_dummies` com `dtype=float` e concatena ao DataFrame (removendo as originais).
   - `ordinal` → aplica `OrdinalEncoder` do scikit-learn (com `unknown_value=-1`).
3. **Escala colunas numéricas** se `SCALE_CFG["enabled"]`:
   - seleciona **apenas** colunas numéricas **contínuas** quando `only_continuous=True` (float/large-range).
   - aplica `StandardScaler` **ou** `MinMaxScaler`.
4. Retorna:
   - `df` (atualizado),
   - `encoding_meta` (categorias vistas, tipo de codificação, colunas excluídas, descartes por cardinalidade),
   - `scaling_meta` (tipo de escala, colunas escaladas, parâmetros do scaler).

---

### 📦 Saídas e Metadados

- **`encoding_meta`** inclui:
  - `encoding`: `"onehot"` **ou** `"ordinal"`
  - `excluded`: lista de colunas ignoradas (ex.: `["Churn","customerID"]`)
  - `high_card_excluded`: colunas **não encodadas** por alta cardinalidade
  - `categorical_columns`: colunas categóricas processadas
  - (para ordinal) `categories_`: categorias aprendidas por coluna

- **`scaling_meta`** inclui:
  - `scaler`: `"standard"` **ou** `"minmax"`
  - `scaled_columns`: lista das colunas escaladas
  - `means_`/`scales_` (para Standard) **ou** `min_`/`range_` (para MinMax) — úteis para **reprodução** no deploy

Esses metadados são importantes para **reaplicar** transformações de forma consistente em produção (ou na inferência).

---

### 🧪 Exemplo de comportamento esperado

Suponha um dataset com colunas:
- `gender` (`Male`/`Female`), `InternetService` (3 categorias), `MonthlyCharges` (float), `tenure` (int), `Churn` (alvo).

Com `onehot` + `standard` e exclusões padrão, o resultado será:
- Novas colunas como `gender_Female`, `gender_Male`, `InternetService_DSL`, etc.
- `MonthlyCharges` escalado (média≈0, desvio≈1), `tenure` **pode** ser ignorado se `only_continuous=True` e for considerado discreto.

---

### 🚧 Armadilhas comuns e proteções do template

- **Explosão de dummies**: colunas com **muitas categorias** são ignoradas (registro em `high_card_excluded`).
- **Vazar o alvo**: `exclude_cols` deve conter a **target** (ex.: `"Churn"`).
- **IDs**: evite codificar/escale IDs (`customerID`); eles não carregam semântica útil.
- **Mixed types**: colunas mal tipadas (número como string) devem ser tratadas antes (use a célula **Qualidade & Tipagem**).

---

### ✅ Status de execução e logs

Ao executar, você verá mensagens no log do tipo:

```
[encode] type=onehot | cols=['gender', 'InternetService'] | high_card_excluded=[]
[scale] method=standard | scaled_cols=['MonthlyCharges']
```

Se o log mostrar `cols=[]`, significa que **não há colunas categóricas** elegíveis (ou foram excluídas por configuração/cardinalidade). Isso é **normal** em alguns datasets.

---

### 🧠 Boas práticas

- Comecar com **one-hot** e **sem escala** para ter uma baseline interpretável.
- Ativar **ordinal** apenas quando existir **ordem natural** nas categorias.
- Escalonar **depois** de separar **treino/validação** para evitar vazamento (no template, esta etapa é opcional e controlada por flag).
- Guardar `encoding_meta` e `scaling_meta` se for levar o modelo para produção.

---

> 💡 **Resumo:**  
> Esta célula converte dados categóricos e numéricos em formatos ideais para modelagem, mantendo controle sobre exclusões, cardinalidade e escala — garantindo robustez, consistência e clareza no tratamento dos dados.

In [17]:
ENCODE_CFG = {
    "enabled":           config.get("encode_categoricals", True),
    "type":              config.get("encoding_type", "onehot"),  # "onehot" | "ordinal"
    "exclude_cols":      ["Churn", "customerID"],
    "high_card_threshold": 50,
}

SCALE_CFG = {
    "enabled":           config.get("scale_numeric", False),
    "method":            config.get("scaler", "standard"),       # "standard" | "minmax"
    "exclude_cols":      ["Churn"],
    "only_continuous":   True,
}

df, encoding_meta, scaling_meta = apply_encoding_and_scaling(
    df, encode_cfg=ENCODE_CFG, scale_cfg=SCALE_CFG
)

2025-10-28 08:46:00,524 | INFO | [encode] type=onehot | cols=[]
2025-10-28 08:46:00,576 | INFO | [scale] method=minmax | cols=['tenure', 'MonthlyCharges', 'TotalCharges']


## 💾 Exportação de Artefatos

Salve dados **intermediários** e **prontos** para uso em modelagem/visualização.  
Também salvamos um **manifest** com metadados das transformações.


In [18]:
# =============================================================================
# 📦 Exportação de Artefatos
# =============================================================================
from datetime import datetime

# Garanta que encoding_meta / scaling_meta existam
encoding_meta = globals().get("encoding_meta", {})
scaling_meta  = globals().get("scaling_meta", {})

manifest: Dict[str, Any] = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "random_seed": RANDOM_SEED,
    "config": config,
    "encoding_meta": encoding_meta,
    "scaling_meta": scaling_meta,
    "memory_mb": float(df.memory_usage(deep=True).sum() / (1024**2)),
    "outlier_flags": [c for c in df.columns if c.endswith("_is_outlier")],
    "imputed_flags": [c for c in df.columns if c.startswith("was_imputed_")],
    "shape": tuple(df.shape),
    "columns": df.columns.tolist(),
}

# Salvar tabelas respeitando a extensão (.csv ou .parquet)
if config.get("export_interim", True):
    save_table(df, OUTPUT_INTERIM)   # usa utils.save_table → respeita a extensão do caminho

if config.get("export_processed", True):
    save_table(df, OUTPUT_PROCESSED)

# Manifest (sempre em JSON)
(ARTIFACTS_DIR).mkdir(parents=True, exist_ok=True)
with open(ARTIFACTS_DIR / "manifest.json", "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)

print("Arquivos gerados:")
print(f"- {OUTPUT_INTERIM if config.get('export_interim', True) else '(pulado)'}")
print(f"- {OUTPUT_PROCESSED if config.get('export_processed', True) else '(pulado)'}")
print(f"- {ARTIFACTS_DIR / 'manifest.json'}")


2025-10-28 08:46:01,918 | INFO | Saved: C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\interim\dataset_interim.csv
2025-10-28 08:46:01,951 | INFO | Saved: C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\processed\dataset_processed.csv
Arquivos gerados:
- C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\interim\dataset_interim.csv
- C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\processed\dataset_processed.csv
- C:\Users\fabio\Projetos DEV\data projects\data-project-template\artifacts\manifest.json


## ✅ Checkpoint

- **Revisar** as colunas derivadas e decisões (imputação, outliers, codificação).  
- **Documentar** no README as escolhas de negócio e justificativas.  

## 📎 Anotações

Esta seção pode ser usada como bloco livre para observações específicas do projeto.
